### Research Process 


#### Init Libraries

In [1]:
import pandas as pd
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler


#### Load Dataset and split into Features and Label

In [2]:
#load data
df=pd.read_csv("breast-cancer-wisconsin-data/data.csv")
#drop irelevent columns for the classification
df = df.drop(columns=['Unnamed: 32', 'id'])
# rearange the data for X - featuers and Y leabels 
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

#### Encode Labels

In [3]:
le = LabelEncoder()
y = le.fit_transform(y)

#### The mapping of the labels

In [4]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'B': 0, 'M': 1}


#### Splitting the data into Train, Test and Valid datasets

In [5]:
#split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#### Scale the data

In [6]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

#### Handling the imbalance between the classes

In [7]:
#handling imbalance data 
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
#the class weight is done only on the train data to impact the learning process and to evaluete beter the model proformence
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))
class_weight_dict

{0: 0.7878787878787878, 1: 1.368421052631579}

as predicted the minorty class 'M'=1 gets higher weight of 1.368

### Neural Networks

##### Init Libraries

In [8]:
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.regularizers import l1, l2
#from keras.layers import Dropout, Flatten, BatchNormalization
from keras import models
from keras import layers
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics
from keras import callbacks
from keras import utils

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report

import tensorflow as tf

tf.config.experimental.enable_op_determinism()

##### Function Helpers

In [9]:
def model_result(model):
    print('================================================================================')
    print(f'Evaluation Report for Model: {model.name}')

    # accuracy
    result = model.evaluate(X_test, y_test, verbose=0)        
    print(f'Loss Value: {result[0]:.3f}, Accuracy: {result[1]*100:.3f}%')

    # confusion matrix
    y_prob = model.predict(X_test, verbose=0)
    y_pred = np.round(y_prob).astype(int)    
    cm_test = confusion_matrix(y_test, y_pred)    
    #sns.heatmap(cm_test,annot=True)
    cm_disp = ConfusionMatrixDisplay(cm_test, display_labels=le.classes_)
    cm_disp.plot()
    plt.show()

    # classification report
    report = classification_report(y_test, y_pred, target_names=le.classes_)
    print(report)

    print('Model Summary:')
    print(model.summary())
    print('================================================================================')

In [10]:
def model_history(history):
    plt.figure(figsize=(15,6))

    val_loss_min_pos = np.argmin(history.history['val_loss'])

    # Plotting the training and validation loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss', color='#8502d1')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Train and Validation Loss')
    plt.plot(val_loss_min_pos, history.history['val_loss'][val_loss_min_pos], 'r*', label='Min Validation Loss')
    plt.text(val_loss_min_pos, history.history['val_loss'][val_loss_min_pos], f'({val_loss_min_pos}, {history.history['val_loss'][val_loss_min_pos]:.3f})', va='bottom')
    plt.legend()

    # Plotting the training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy', color='#8502d1')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.plot(val_loss_min_pos, history.history['val_accuracy'][val_loss_min_pos], 'r*', label='Validation Accuracy @ Min Validation Loss')
    plt.text(val_loss_min_pos, history.history['val_accuracy'][val_loss_min_pos], f'({val_loss_min_pos}, {history.history['val_accuracy'][val_loss_min_pos]:.3f})', va='bottom')
    plt.title('Train and Validation Accuracy')
    plt.legend()

    plt.show()    

In [11]:
def proc_history(history):

    val_loss_min_pos = np.argmin(history.history['val_loss'])

    train_acc = history.history['accuracy'][val_loss_min_pos]
    train_loss = history.history['loss'][val_loss_min_pos]

    val_acc = history.history['val_accuracy'][val_loss_min_pos]
    val_loss = history.history['val_loss'][val_loss_min_pos]

    return {'Train Accuracy': train_acc, 'Train Loss': train_loss, 'Validation Accuracy': val_acc, 'Validation Loss': val_loss}

In [68]:
def model_fit(model, epochNum, pca=False, verbose='auto'):    
    utils.set_random_seed(0)
    
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=40, restore_best_weights=True, verbose=verbose)
    #reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001, verbose=verbose)
    checkpoint_filepath = f'model_checkpoints/{model.name}_checkpoint.model.keras'
    model_checkpoint_callback = callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        monitor='val_loss',
        mode='min',
        save_best_only=True,
        verbose=verbose)

    model.compile(optimizer=optimizers.Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train, y_train, batch_size=50, epochs=epochNum, validation_data=(X_val, y_val), class_weight=class_weight_dict, callbacks=[model_checkpoint_callback, early_stopping], verbose=verbose)

    model = models.load_model(checkpoint_filepath)

    return history, model

##### Build the model

In [47]:
models_list = []

### SLP

In [48]:
# SLP
model = models.Sequential(name=f'SLP_1')
model.add(layers.Dense(1, activation='sigmoid', input_shape=(30,)))
models_list.append(model)

# Add Dropout 0.1, 0.2
model = models.Sequential(name=f'SLP_2')
model.add(layers.Dropout(rate=0.1, input_shape=(30,)))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'SLP_3')
model.add(layers.Dropout(rate=0.2, input_shape=(30,)))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

d:\Yakir\Development\Python\DL\Breast-Cancer-Prediction\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\Yakir\Development\Python\DL\Breast-Cancer-Prediction\.venv\Lib\site-packages\keras\src\layers\regularization\dropout.py:42: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### MLP

In [49]:
# MLP 1 Layer
model = models.Sequential(name=f'MLP_1')
model.add(layers.Dense(2, activation='relu', input_shape=(30,)))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_2')
model.add(layers.Dense(5, activation='relu', input_shape=(30,)))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_3')
model.add(layers.Dense(10, activation='relu', input_shape=(30,)))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_4')
model.add(layers.Dense(15, activation='relu', input_shape=(30,)))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_5')
model.add(layers.Dense(30, activation='relu', input_shape=(30,)))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

In [50]:
# MLP 2 Layer
model = models.Sequential(name=f'MLP_6')
model.add(layers.Dense(2, activation='relu', input_shape=(30,)))
model.add(layers.Dense(2, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid')), models_list.append(model)

model = models.Sequential(name=f'MLP_7')
model.add(layers.Dense(5, activation='relu', input_shape=(30,)))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_8')
model.add(layers.Dense(10, activation='relu', input_shape=(30,)))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_9')
model.add(layers.Dense(15, activation='relu', input_shape=(30,)))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_10')
model.add(layers.Dense(30, activation='relu', input_shape=(30,)))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

In [51]:
# MLP 3 Layer
model = models.Sequential(name=f'MLP_11')
model.add(layers.Dense(30, activation='relu', input_shape=(30,)))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_12')
model.add(layers.Dense(10, activation='relu', input_shape=(30,)))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

### MLP - Dropout 0.3

In [52]:
# MLP 1 Layer
model = models.Sequential(name=f'MLP_13')
model.add(layers.Dense(2, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_14')
model.add(layers.Dense(5, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_15')
model.add(layers.Dense(10, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_16')
model.add(layers.Dense(15, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_17')
model.add(layers.Dense(30, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

In [53]:
# MLP 2 Layer
model = models.Sequential(name=f'MLP_18')
model.add(layers.Dense(2, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(2, activation='relu'))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(1, activation='sigmoid')), models_list.append(model)

model = models.Sequential(name=f'MLP_19')
model.add(layers.Dense(5, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_20')
model.add(layers.Dense(10, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_21')
model.add(layers.Dense(15, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_22')
model.add(layers.Dense(30, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

In [54]:
# MLP 3 Layer
model = models.Sequential(name=f'MLP_23')
model.add(layers.Dense(30, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_24')
model.add(layers.Dense(10, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dropout(rate=0.3))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

### MLP - Dropout 0.5

In [55]:
# MLP 1 Layer
model = models.Sequential(name=f'MLP_25')
model.add(layers.Dense(2, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_26')
model.add(layers.Dense(5, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_27')
model.add(layers.Dense(10, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_28')
model.add(layers.Dense(15, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_29')
model.add(layers.Dense(30, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

In [56]:
# MLP 2 Layer
model = models.Sequential(name=f'MLP_30')
model.add(layers.Dense(2, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(2, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid')), models_list.append(model)

model = models.Sequential(name=f'MLP_31')
model.add(layers.Dense(5, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_32')
model.add(layers.Dense(10, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_33')
model.add(layers.Dense(15, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_34')
model.add(layers.Dense(30, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

In [57]:
# MLP 3 Layer
model = models.Sequential(name=f'MLP_35')
model.add(layers.Dense(30, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

model = models.Sequential(name=f'MLP_36')
model.add(layers.Dense(10, activation='relu', input_shape=(30,)))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(1, activation='sigmoid'))
models_list.append(model)

In [58]:
models_list

[<Sequential name=SLP_1, built=True>,
 <Sequential name=SLP_2, built=True>,
 <Sequential name=SLP_3, built=True>,
 <Sequential name=MLP_1, built=True>,
 <Sequential name=MLP_2, built=True>,
 <Sequential name=MLP_3, built=True>,
 <Sequential name=MLP_4, built=True>,
 <Sequential name=MLP_5, built=True>,
 <Sequential name=MLP_6, built=True>,
 <Sequential name=MLP_7, built=True>,
 <Sequential name=MLP_8, built=True>,
 <Sequential name=MLP_9, built=True>,
 <Sequential name=MLP_10, built=True>,
 <Sequential name=MLP_11, built=True>,
 <Sequential name=MLP_12, built=True>,
 <Sequential name=MLP_13, built=True>,
 <Sequential name=MLP_14, built=True>,
 <Sequential name=MLP_15, built=True>,
 <Sequential name=MLP_16, built=True>,
 <Sequential name=MLP_17, built=True>,
 <Sequential name=MLP_18, built=True>,
 <Sequential name=MLP_19, built=True>,
 <Sequential name=MLP_20, built=True>,
 <Sequential name=MLP_21, built=True>,
 <Sequential name=MLP_22, built=True>,
 <Sequential name=MLP_23, built=True>

In [69]:
from time import time

models_df = pd.DataFrame(columns=['Model Name', 'Train Accuracy', 'Train Loss', 'Validation Accuracy', 'Validation Loss', 'Test Accuracy', 'Test Loss'])
print('Processing Models:')
for model in models_list:
    print(f'Fitting {model.name}...', end= " ")    

    t_start = time()    
    history, model = model_fit(model, 2000, verbose=0)
    t_end = time()

    proc_data = proc_history(history)  

    eval_res = model.evaluate(X_test, y_test, verbose=0)  

    new_row = {'Model Name': model.name, **proc_data, 'Test Accuracy': eval_res[1], 'Test Loss': eval_res[0]}
    models_df.loc[len(models_df)] = new_row

    print(f'Done! ({t_end-t_start} [s])')

Processing Models:
Fitting SLP_1... Done! (2.7189948558807373 [s])
Fitting SLP_2... Done! (3.126000165939331 [s])
Fitting SLP_3... Done! (3.1361300945281982 [s])
Fitting MLP_1... Done! (2.856057643890381 [s])
Fitting MLP_2... Done! (2.770138740539551 [s])
Fitting MLP_3... Done! (2.8081390857696533 [s])
Fitting MLP_4... Done! (2.8282053470611572 [s])
Fitting MLP_5... Done! (2.81964373588562 [s])
Fitting MLP_6... Done! (2.997732639312744 [s])
Fitting MLP_7... Done! (2.9592044353485107 [s])
Fitting MLP_8... Done! (2.9713380336761475 [s])
Fitting MLP_9... Done! (3.035731315612793 [s])
Fitting MLP_10... Done! (2.98618483543396 [s])
Fitting MLP_11... Done! (3.198922872543335 [s])
Fitting MLP_12... Done! (3.2087631225585938 [s])
Fitting MLP_13... Done! (3.0255067348480225 [s])
Fitting MLP_14... Done! (2.851266622543335 [s])
Fitting MLP_15... Done! (3.021934747695923 [s])
Fitting MLP_16... Done! (2.8589487075805664 [s])
Fitting MLP_17... Done! (2.9198501110076904 [s])
Fitting MLP_18... Done! (

In [26]:
models_df

,Model Name,Train Accuracy,Train Loss,Validation Accuracy,Validation Loss
0,SLP_1,0.989011,0.057050,0.967033,0.130379
1,SLP_2,0.975275,0.076719,0.967033,0.127834
2,SLP_3,0.969780,0.098340,0.967033,0.129408
3,MLP_1,0.978022,0.063895,0.967033,0.151033
4,MLP_2,0.983516,0.063604,0.956044,0.129547
5,MLP_3,0.980769,0.056463,0.956044,0.133248
6,MLP_4,0.986264,0.049312,0.967033,0.125480
7,MLP_5,0.983516,0.055378,0.967033,0.119391
8,MLP_6,0.991758,0.100684,0.956044,0.234191
9,MLP_7,0.980769,0.063793,0.967033,0.119832


In [38]:
models_df.sort_values(ascending=[False, True], by=['Validation Accuracy', 'Validation Loss']).head()

,Model Name,Train Accuracy,Train Loss,Validation Accuracy,Validation Loss
18,MLP_16,0.978022,0.061854,0.967033,0.112554
30,MLP_28,0.969780,0.097536,0.967033,0.118470
20,MLP_18,0.802198,0.356862,0.967033,0.119138
7,MLP_5,0.983516,0.055378,0.967033,0.119391
9,MLP_7,0.980769,0.063793,0.967033,0.119832


In [42]:
models_df

,Model Name,Train Accuracy,Train Loss,Validation Accuracy,Validation Loss,Test Loss,Test Accuracy
0,SLP_1,0.989011,0.063783,0.967033,0.124730,0.078172,0.95614
1,SLP_2,0.980769,0.077067,0.967033,0.127795,0.086853,0.95614
2,SLP_3,0.961538,0.092649,0.967033,0.132687,0.082823,0.95614


In [46]:
models_df

,Model Name,Train Accuracy,Train Loss,Validation Accuracy,Validation Loss,Test Accuracy,Test Loss
0,SLP_1,0.989011,0.063556,0.967033,0.125773,0.95614,0.077795
1,SLP_2,0.969780,0.082634,0.967033,0.124347,0.95614,0.076808
2,SLP_3,0.980769,0.084145,0.967033,0.126178,0.95614,0.072279


In [63]:
models_df.sort_values(ascending=[False, True], by=['Validation Accuracy', 'Validation Loss'])

,Model Name,Train Accuracy,Train Loss,Validation Accuracy,Validation Loss,Test Accuracy,Test Loss
37,MLP_35,0.936813,0.213505,0.967033,0.115124,0.964912,0.090839
16,MLP_14,0.947802,0.122453,0.967033,0.117187,0.973684,0.072930
18,MLP_16,0.980769,0.068708,0.967033,0.120926,0.956140,0.088397
11,MLP_9,0.986264,0.050299,0.967033,0.121068,0.973684,0.069267
4,MLP_2,0.986264,0.047493,0.967033,0.123979,0.973684,0.076535
15,MLP_13,0.876374,0.236691,0.967033,0.124363,0.973684,0.073229
28,MLP_26,0.964286,0.141056,0.967033,0.127005,0.956140,0.087613
0,SLP_1,0.983516,0.062885,0.967033,0.127054,0.964912,0.075304
14,MLP_12,0.980769,0.051441,0.967033,0.127158,0.947368,0.070626
35,MLP_33,0.920330,0.160369,0.967033,0.130773,0.964912,0.088441


In [64]:
models_df.sort_values(ascending=False, by='Test Accuracy')

,Model Name,Train Accuracy,Train Loss,Validation Accuracy,Validation Loss,Test Accuracy,Test Loss
27,MLP_25,0.813187,0.356778,0.945055,0.238595,0.982456,0.175749
38,MLP_36,0.956044,0.177596,0.956044,0.154935,0.973684,0.101945
15,MLP_13,0.876374,0.236691,0.967033,0.124363,0.973684,0.073229
26,MLP_24,0.939560,0.165762,0.967033,0.138537,0.973684,0.068869
11,MLP_9,0.986264,0.050299,0.967033,0.121068,0.973684,0.069267
10,MLP_8,0.975275,0.069972,0.956044,0.119801,0.973684,0.072247
16,MLP_14,0.947802,0.122453,0.967033,0.117187,0.973684,0.072930
4,MLP_2,0.986264,0.047493,0.967033,0.123979,0.973684,0.076535
33,MLP_31,0.887363,0.305434,0.967033,0.151435,0.964912,0.092112
17,MLP_15,0.969780,0.083142,0.967033,0.136200,0.964912,0.083680


In [67]:
models_df

,Model Name,Train Accuracy,Train Loss,Validation Accuracy,Validation Loss,Test Accuracy,Test Loss
0,SLP_1,0.983516,0.060272,0.967033,0.127216,0.964912,0.073434
1,SLP_2,0.986264,0.070067,0.967033,0.127630,0.973684,0.073539
2,SLP_3,0.972527,0.079117,0.967033,0.124747,0.964912,0.068795
3,MLP_1,0.994505,0.037767,0.956044,0.126939,0.956140,0.091871
4,MLP_2,0.986264,0.047196,0.967033,0.124598,0.973684,0.076577
5,MLP_3,0.986264,0.067250,0.956044,0.127990,0.964912,0.102415
6,MLP_4,0.989011,0.038758,0.956044,0.122287,0.964912,0.064709
7,MLP_5,0.983516,0.060775,0.967033,0.139149,0.956140,0.086962
8,MLP_6,0.978022,0.061742,0.967033,0.146914,0.964912,0.063948
9,MLP_7,0.980769,0.066981,0.967033,0.150776,0.956140,0.087910
